In [316]:
### SETTING UP DIRECTORIES

import os
import sys
import warnings

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from mizani.formatters import percent_format
from plotnine import *
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse, rmse

warnings.filterwarnings("ignore")

# set working directory for da_data_repo -- replace the
os.chdir('C:/Users/45919/Desktop/CEU/Machine Learning')


In [188]:
from stargazer import stargazer

In [8]:
# load dataset (as unicode, to avoid size and memory warnings)

df = pd.read_csv("morg-2014-emp.csv", header=0, sep=','
)

C:\Users\45919\AppData\Local\Temp/ipykernel_23548/2823244067.py:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.


In [9]:
#Elementary and middle school teachers is selected. Keep only the observations that has the chosen profession and drop others.

df=df[df['occ2012']==2310]

In [24]:
df.isna().sum()

Unnamed: 0       0
hhid             0
intmonth         0
stfips           0
weight           0
earnwke          0
uhours           0
grade92          0
race             0
ethnic        3375
age              0
sex              0
marital          0
ownchild         0
chldpres         0
prcitshp         0
state            0
ind02            0
occ2012          0
class            0
unionmme         0
unioncov      1861
lfsr94           0
w                0
lnw              0
agesq            0
dtype: int64

Majority of observations did not indicate ethnicity, so I will not include this variable in the models. Union coverage column also has missing observations for half of the dataset which is why I will use only union membership instead.

In [146]:
# CREATE THE NECESSARY VARIABLES 

df["w"] = df["earnwke"] / df["uhours"]
df["agesq"] = np.power(df["age"], 2)

In [147]:
#Describe the data to check if the filtering was successful and understand the descriptive stats
df.loc[df['w']>=1,['earnwke', 'uhours','w']].describe()

,earnwke,uhours,w
count,3632.000000,3632.000000,3632.000000
mean,1035.332084,40.690253,25.939639
std,528.015992,8.730560,18.590945
min,28.000000,1.000000,1.602500
25%,692.300000,40.000000,16.826750
50%,961.000000,40.000000,23.076750
75%,1250.000000,40.000000,31.250000
max,2884.610000,80.000000,583.330000


## Linear Regression Models

In [248]:
model1="w~age + agesq + sex"
model2="w~age + agesq + sex + grade92 + marital"
model3="w~age + agesq + sex + grade92 + marital + chldpres + race +prcitshp + unionmme + lfsr94"
model4="w~age + agesq + sex + grade92 + marital + chldpres + race +prcitshp + unionmme + lfsr94 + ind02 + ownchild + uhours + weight + hhid"

models = [model1,model2,model3,model4]

I wanted to include state variable as one of the predictors, but computer capacity could not handle that many variables. It was creating too many dummies. So, I just excluded state from analysis even though it is a very important predictor of wage since wage rate varies from state to state.

In [249]:
regs = []
for mod in models:
    regs.append(smf.ols(mod, df).fit(cov_type="HC1"))

In [250]:
stargazer.Stargazer(regs)

In [251]:
# BIC in the full sample for each model

BIC = []
for bics in regs:
    BIC.append(bics.bic)
BIC = [float('{:.3f}'.format(i)) for i in BIC]
BIC

[31492.927, 31430.292, 31461.914, 31437.16]

In [252]:
# RMSE in the full sample for each model
wage = df["w"].values.tolist()
RMSE = []

#def rmse(predictions, targets):
 #   return np.sqrt(((predictions - targets) ** 2).mean())

for rmses in regs:
    w_pred = rmses.predict()
    RMSE.append(rmse(wage, w_pred))
RMSE = [float('{:.3f}'.format(i)) for i in RMSE]
RMSE

[18.307, 18.109, 18.024, 18.126]

### Cross Validation

In [253]:
import statsmodels.formula.api as smf
from sklearn.model_selection import KFold
from statsmodels.tools.eval_measures import rmse

In [254]:
def ols_crossvalidator(formula:str, data: pd.DataFrame, n_folds=5, average_rmse=True):


    # Get dependent variable

    y = formula.split("~")[0].strip()

    # Get statistics on the whole work data

    model = smf.ols(formula, data=data).fit()

    rsquared = model.rsquared
    # n_coefficients = model.params.shape[0]
    n_coefficients = (
        model.df_model + 1
    )  # This might differ from model.params.shape[0], because of collinear predictors
    bic = model.bic
    rmse_alldata = rmse(model.predict(), data[y])

    # Calculating test and train RMSE-s for each fold

    k = KFold(n_splits=n_folds, shuffle=False, random_state=None)

    rmse_train = []
    rmse_test = []

    for train_index, test_index in k.split(data):

        data_train, data_test = data.iloc[train_index, :], data.iloc[test_index, :]

        model = smf.ols(formula, data=data_train).fit()

        rmse_train.append(rmse(data_train[y], model.predict(data_train)))
        rmse_test.append(rmse(data_test[y], model.predict(data_test)))

    if average_rmse:
        rmse_train = np.mean(rmse_train)
        rmse_test = np.mean(rmse_test)

    return {
        "RMSE": rmse_alldata,
        "R-squared": rsquared,
        "BIC": bic,
        "Coefficients": n_coefficients,
        "Training RMSE": rmse_train,
        "Test RMSE": rmse_test,
    }

In [255]:
n_fold = 4

In [256]:
cv_l = []
for mod in models:
    cv_l.append(ols_crossvalidator(mod, df, n_fold, average_rmse=False))

In [257]:
(
    pd.DataFrame(cv_l)
    .round(3)
    .assign(
        RMSE=lambda x: x["RMSE"],
        BIC=lambda x: x["BIC"].astype(int),
        Coefficients=lambda x: x["Coefficients"].astype(int),
        Model=["Model " + str(i + 1) for i in range(len(models))],
    )
    .filter(["Model", "Coefficients", "R-squared", "RMSE", "BIC"])
    .set_index("Model")
)

,Coefficients,R-squared,RMSE,BIC
Model,,,,
Model 1,4,0.031,18.307,31492
Model 2,6,0.052,18.109,31430
Model 3,14,0.061,18.024,31461
Model 4,6,0.050,18.126,31437


#### Wages of elementary and middle school teachers estimated and evaluated using 4-fold cross-validation and RMSE

In [258]:
pd.DataFrame(
    [cv["Test RMSE"] for cv in cv_l],
    index=["Model " + str(i + 1) for i in range(len(cv_l))],
    columns=["Fold" + str(i + 1) for i in range(len(cv_l[0]["Test RMSE"]))],
).assign(Average=lambda x: x.mean(axis=1)).T.round().astype(int)

,Model 1,Model 2,Model 3,Model 4
Fold1,12,11,11,13
Fold2,23,23,23,22
Fold3,22,22,22,22
Fold4,14,13,14,14
Average,18,17,17,18


Since the models lack some of the most important factors that affect wage such as geographic location, experience and skills, they are producing quite similar results because the basic models 1 and 2 already include the major determinants of wage.

### Prediction

Now I will take one observation from the dataset and try to predict it using point_predict_with_conf_int function from gabors-data-analysis book.

In [318]:
df = df.loc[
    :,
    [
        "age",
        "agesq",
        "sex" ,
        "grade92",
        "marital",
        "chldpres",
        "race",
        "prcitshp",
        "unionmme",
        "lfsr94",
        "w",
    ],
]

In [319]:
df.dtypes

age           int64
agesq         int64
sex           int64
grade92       int64
marital       int64
chldpres      int64
race          int64
prcitshp     object
unionmme     object
lfsr94       object
w           float64
dtype: object

In [320]:
#To see the values of only the necessary columns, I dropped other columns because some columns in the middle 
#were not shown previously.
df=df[['age','agesq','sex','grade92','marital','chldpres','race','prcitshp','unionmme','lfsr94','w',]]
df.head(10)

,age,agesq,sex,grade92,marital,chldpres,race,prcitshp,unionmme,lfsr94,w
21,31,961,2,44,7,0,1,"Native, Born In US",No,Employed-At Work,20.673000
30,21,441,2,40,1,0,1,"Native, Born In US",No,Employed-At Work,8.500000
50,39,1521,2,43,1,0,1,"Native, Born In US",Yes,Employed-At Work,14.957111
84,57,3249,2,44,7,0,2,"Native, Born In US",Yes,Employed-At Work,17.857143
130,45,2025,2,39,1,9,1,"Native, Born In US",Yes,Employed-At Work,13.000000
145,32,1024,2,43,1,5,1,"Native, Born In US",Yes,Employed-At Work,15.384600
146,35,1225,1,43,1,5,1,"Native, Born In US",Yes,Employed-At Work,22.307600
162,56,3136,2,43,1,0,1,"Native, Born In US",Yes,Employed-At Work,19.230714
165,56,3136,2,40,7,0,2,"Native, Born In US",Yes,Employed-At Work,11.000000
171,25,625,2,44,7,0,1,"Native, Born In US",Yes,Employed-At Work,25.000000


I will enter the values of variables for observation 21 as a new data point, predict its wage and compare the result with actual wage of observation 21. 

In [332]:
new = pd.DataFrame(
    pd.Series(
        {  
        "age": 31,
        "agesq": 961,
        "sex": 2 ,
        "grade92": 44,
        "marital": 7,
        "chldpres": 0,
        "race": 1,
        "prcitshp": "Native, Born In US",
        "unionmme": "No",
        "lfsr94": "Employed-At Work",
        "w": np.nan,
        }
    )
).T
new

,age,agesq,sex,grade92,marital,chldpres,race,prcitshp,unionmme,lfsr94
0,31,961,2,44,7,0,1,"Native, Born In US",No,Employed-At Work


In [333]:
new.append(new)

,age,agesq,sex,grade92,marital,chldpres,race,prcitshp,unionmme,lfsr94
0,31,961,2,44,7,0,1,"Native, Born In US",No,Employed-At Work
0,31,961,2,44,7,0,1,"Native, Born In US",No,Employed-At Work


In [334]:
reg1 = regs[0]
reg3 = regs[2]
reg4 = regs[3]

In [335]:
import statsmodels


def point_predict_with_conf_int(
    regression: statsmodels.regression.linear_model.RegressionResultsWrapper,
    new_datapoint: pd.DataFrame,
    interval_precision=0.95,
    round_n=2,
) -> dict:
    """
    Does point prediction and interval prediction for a new datapoint.
        Parameters
    ----------
    regression : statsmodels.regression.linear_model.RegressionResultsWrapper
        Fitted regression model.
    new_datapoint : pd.DataFrame
        Database containing a new observation.
    interval_precision : float, default=0.95
        Precision of interval prediction.
    round_n: int, default=2
        Decimals to round floats in output.
    """

    summaryframe = regression.get_prediction(new_datapoint).summary_frame(
        alpha=1 - interval_precision
    )

    point_prediction = round(summaryframe["mean"].values[0], round_n)

    conf_int = [
        round(i, round_n)
        for i in summaryframe[["obs_ci_lower", "obs_ci_upper"]].values[0]
    ]

    if round_n == 0:
        point_prediction = int(point_prediction)
        conf_int = [int(i) for i in conf_int]
    else:
        pass

    return {
        "Point prediction": point_prediction,
        f"Prediction Interval ({round(interval_precision*100)}%)": conf_int,
    }


class Error(Exception):
    """Base class for other exceptions"""

    pass


class ConfintError(Error):
    """
    Error raised when a confidence interval
    does not match with required format.
    """

    def __init__(
        self,
        message="Confidence intervals are two numbers, so len(conf_int) must be 2.",
    ):
        self.message = message
        super().__init__(self.message)


In [336]:
p95 = pd.DataFrame(
    [
        point_predict_with_conf_int(reg1, new, interval_precision=0.95, round_n=0),
        point_predict_with_conf_int(reg3, new, interval_precision=0.95, round_n=0),
        
    ],
    index=["Model 1", "Model 3"],
).T
p95.loc["Prediction Interval (95%)", :] = p95.loc["Prediction Interval (95%)", :].apply(
    format_confidence_interval
)

AttributeError: 'DataFrame' object has no attribute 'dtype'